In [1]:
import numpy as np
import math 
import queue
import os
import neat
import visualize
import random

import GameManager
from GameManager import map_manager, Tile, Unit, Team

import Neat_AI
from Neat_AI import eval_genomes, run

In [2]:
local_dir = os.path.abspath('')
config_path = os.path.join(local_dir, 'config-feedforward')

In [153]:
dimensions = (5,5)
MapManager = map_manager(dimensions)

pos1 = (0, 0)
pos2 = (4, 4)
unit1 = MapManager.place_unit(pos1, 0)
unit2 = MapManager.place_unit(pos2, 1)

print(MapManager)

#MapManager.find_movement(unit1)
print(MapManager)

In [36]:
dimensions = (8,8)
manager = map_manager(dimensions)

pos_pick = random.randint(0, 1)
pos_1, pos_2, pos_3, pos_4, pos_5, pos_6 = (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)
if (pos_pick == 0):
    #Team 0 units between 0 and 2 in the x, full length in the y
    pos_1 = (random.randint(0, (dimensions[1]//3)-1), random.randint(0, 2))
    pos_2 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(0, 2))    
    pos_3 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(0, 2))     
    #Team 1 units between 5 and 7 in the x, one half of y      
    pos_4 = (random.randint(0, (dimensions[1]//3)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_5 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_6 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(dimensions[0]-3, dimensions[0]-1))
elif (pos_pick == 1):
    #Team 0 units between 5 and 7 in the x, full length in the y
    pos_1 = (random.randint(0, (dimensions[1]//3)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_2 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_3 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    #Team 1 units between 0 and 2 in the x, one half of y
    pos_4 = (random.randint(0, (dimensions[1]//3)-1), random.randint(0, 2))
    pos_5 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(0, 2))
    pos_6 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(0, 2))

manager.reset_map()
unit0_1 = manager.place_unit(pos_1, 0)
unit0_2 = manager.place_unit(pos_2, 0)
unit0_3 = manager.place_unit(pos_3, 0)

unit1_1 = manager.place_unit(pos_4, 1)
unit1_2 = manager.place_unit(pos_5, 1)
unit1_3 = manager.place_unit(pos_6, 1)
print(manager)

input_list = list(np.zeros((15)))
index = 0
#Allied pieces input
for i in range(len(my_units)):
    c_unit = my_units[i]
    if c_unit == unit:  #Input for selected unit entered at beginning
        continue
    input_list[index] = (c_unit.pos[0] - unit0_1.pos[0])/(dimensions[0]-1)
    input_list[index+1] = (c_unit.pos[1] - unit0_1.pos[1])/(dimensions[1]-1)
    input_list[index+2] = (c_unit.temp_hp / 100.0)
    index += 3

#Opponent pieces input
for i in range(len(op_units)):
    c_unit = op_units[i]
    input_list[index] = (c_unit.pos[0] - unit0_1.pos[0])/(dimensions[0]-1)
    input_list[index+1] = (c_unit.pos[1] - unit0_1.pos[1])/(dimensions[1]-1)
    input_list[index+2] = (c_unit.temp_hp / 100.0)
    index += 3
    
print(unit0_1)
    
format_in = [ '%.2f' % elem for elem in input_tup ]
print("input vector: " + str(format_in))

[ ][ ][ ][ ][ ][ ][ ][ ]
[ ][1][ ][ ][ ][0][ ][ ]
[1][ ][ ][ ][ ][ ][0][ ]
[ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][0]
[ ][ ][ ][ ][ ][ ][ ][ ]
[1][ ][ ][ ][ ][ ][ ][ ]

Unit:
	pos: (1, 5)
	curr_move: 4
	hp: 100
	temp_hp: 100
	max_move: 4
	Att: 20
	Def: 10
	Team: 0

input vector: ['0.20', '0.07', '-0.43', '-0.60', '0.07', '-0.43', '0.20', '0.00', '-0.43', '-0.60', '0.00', '-0.43', '-0.60', '0.07', '-0.36', '-0.60']


In [31]:
dimensions = (15,15)
manager = map_manager(dimensions)
#win_net1
#win_net2

pos_pick = random.randint(0, 1)
pos_1, pos_2, pos_3, pos_4, pos_5, pos_6 = (0, 0),(0, 0),(0, 0),(0, 0),(0, 0),(0, 0)
if (pos_pick == 0):
    #Team 0 units between 0 and 2 in the x, full length in the y
    pos_1 = (random.randint(0, (dimensions[1]//3)-1), random.randint(0, 2))
    pos_2 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(0, 2))    
    pos_3 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(0, 2))     
    #Team 1 units between 5 and 7 in the x, one half of y      
    pos_4 = (random.randint(0, (dimensions[1]//3)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_5 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_6 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(dimensions[0]-3, dimensions[0]-1))
elif (pos_pick == 1):
    #Team 0 units between 5 and 7 in the x, full length in the y
    pos_1 = (random.randint(0, (dimensions[1]//3)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_2 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    pos_3 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(dimensions[0]-3, dimensions[0]-1))
    #Team 1 units between 0 and 2 in the x, one half of y
    pos_4 = (random.randint(0, (dimensions[1]//3)-1), random.randint(0, 2))
    pos_5 = (random.randint(dimensions[1]//3, ((dimensions[1]//3)*2)-1), random.randint(0, 2))
    pos_6 = (random.randint(((dimensions[1]//3)*2), dimensions[1]-1), random.randint(0, 2))

manager.reset_map()
unit0_1 = manager.place_unit(pos_1, 0)
unit0_2 = manager.place_unit(pos_2, 0)
unit0_3 = manager.place_unit(pos_3, 0)

unit1_1 = manager.place_unit(pos_4, 1)
unit1_2 = manager.place_unit(pos_5, 1)
unit1_3 = manager.place_unit(pos_6, 1)

print(manager)

units0 = [unit0_1, unit0_2, unit0_3]
units1 = [unit1_1, unit1_2, unit1_3]

my_units = []
op_units = []
net = None
#print("GENOME EVALUATION: ")
while (manager.game_result() == -1 and manager.turn_count < 7): #Turn Count limit may have to be modified
    if manager.curr_team == 0:
        my_units = units0
        op_units = units1
        net = win_net1
    else:
        my_units = units1
        op_units = units0
        net = win_net2

    for unit in manager.Teams[manager.curr_team].units:
        move_list = manager.find_movement(unit)
        input_list = list(np.zeros((16)))

        win_move = (0, 0)
        win_weight = -float("inf")
        for move in move_list:
            #Cannot stay in the same spot: avoid getting trapped in a 'hole'
            if (move.pos == unit.pos):
                pass
            saved_pos = unit.pos    #save unit pos for later
            unit.pos = move.pos

            if move.is_attack:      #move is of Tile Class
                move_pos = move.move_parent.pos
                manager.sim_combat(unit, move.unit_ref) 

            index = 0
            #This unit's input is always first
            input_list[index] = (unit.temp_hp / 100.0)
            index += 1

            #Allied pieces input
            for i in range(len(my_units)):
                c_unit = my_units[i]
                if c_unit == unit:  #Input for selected unit entered at beginning
                    continue
                input_list[index] = (c_unit.pos[0] - move.pos[0])/(dimensions[0]-1)
                input_list[index+1] = (c_unit.pos[1] - move.pos[1])/(dimensions[1]-1)
                input_list[index+2] = (c_unit.temp_hp / 100.0)
                index += 3

           #Opponent pieces input
            for i in range(len(op_units)):
                c_unit = op_units[i]
                input_list[index] = (c_unit.pos[0] - move.pos[0])/(dimensions[0]-1)
                input_list[index+1] = (c_unit.pos[1] - move.pos[1])/(dimensions[1]-1)
                input_list[index+2] = (c_unit.temp_hp / 100.0)
                index += 3
                
            #VERY IMPORTANT, must be executed after SIMULATED combat!!!
            if (move.is_attack):
                manager.reset_temp_hp() 

            input_tup = tuple(input_list)
            format_in = [ '%.2f' % elem for elem in input_tup ]
            print("input vector: " + str(format_in))

            output = net.activate(input_tup)

            format_out = [ '%.2f' % elem for elem in output ]
            #print("output vector: " + str(format_out))

            #print(len(output[0]))
            #print(len(win_weight))
            if (output[0] >= win_weight):
                win_move = move.pos
                win_weight = output[0]

            unit.pos = saved_pos    #revert back to saved pos

        #print("unit 1 pos: " + str(unit1.pos))
        #print("unit 2 pos: " + str(unit2.pos))

        #print("move: " + str(move))
        manager.move_unit(unit, win_move)
        print(manager)

    #print("new_pos: "+ str(unit1.pos))

    #Next Turn
    manager.Turn()

print(manager.game_result())

[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[0][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][1]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][0][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][1][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][1][ ][ ]
[ ][ ][0][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]

input vector: ['1.00', '0.21', '0.07', '1.00', '0.64', '0.14', '1.00', '0.00', '1.00', '1.00', '0.50', '0.86', '1.00', '0.57', '0.86', '1.00']
input vector: ['1.00', '0.14', '0.07', '1.00', '0.57', '0.14', '1.00', '-0.07', '1.00', '1.00', '0.43', '0.86', '1.00', '0.50', '0.86', '1.00']
input vector: ['1.00',

In [9]:
from importlib import reload
reload(GameManager)
reload(Neat_AI)

from GameManager import map_manager, Tile, Unit, Team
from Neat_AI import eval_genomes, run

In [10]:
win_net = run(config_path)

	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_conn


 ****** Running generation 0 ****** 

defaulting to this genome
Population's average fitness: 0.00067 stdev: 0.00814
Best fitness: 0.10000 - size: (3, 34) - species 2 - id 47
Average adjusted fitness: 0.001
Mean genetic distance 3.231, standard deviation 0.790
Population of 150 members in 6 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    39      0.0    0.000     0
     2    0    84      0.1    0.002     0
     3    0    13      0.0    0.000     0
     4    0     4      0.0    0.000     0
     5    0     2       --       --     0
     6    0     8       --       --     0
Total extinctions: 0
Generation time: 35.346 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.01200 stdev: 0.06101
Best fitness: 0.50000 - size: (2, 15) - species 1 - id 172
Average adjusted fitness: 0.007
Mean genetic distance 2.856, standard deviation 0.828
Population of 151 members in 6 species:
   ID   age  size  fitness  adj fit 

Population's average fitness: 0.07333 stdev: 0.14360
Best fitness: 0.60000 - size: (2, 12) - species 6 - id 1722
Average adjusted fitness: 0.023
Mean genetic distance 2.456, standard deviation 0.756
Population of 150 members in 6 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   12    35      0.5    0.041     9
     2   12     4      0.0    0.000    11
     3   12    15      0.0    0.000     9
     4   12     7      0.0    0.000     9
     5   12     2      0.0    0.000    11
     6   12    87      0.6    0.099     4
Total extinctions: 0
Generation time: 26.631 sec (29.675 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.07667 stdev: 0.16428
Best fitness: 0.70000 - size: (3, 13) - species 6 - id 1859
Average adjusted fitness: 0.025
Mean genetic distance 2.517, standard deviation 0.800
Population of 151 members in 6 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  =

In [11]:
saved_net = win_net

In [12]:
win_net1 = saved_net

In [13]:
win_net2 = saved_net